In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import io
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import pdb

BATCH_SIZE = 32

In [3]:
device = torch.device("cuda:0")
print(device)

cuda:0


### Helper functions for each step in the pipeline

In [4]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    print(n, d)
    i = 0
    for line in fin:
        if i == VOCAB_SIZE:
            break
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
        i += 1
    return data

In [5]:
from collections import Counter
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1
VOCAB_SIZE = 50000

def build_vocab():
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    word_vectors = pkl.load(open("fasttext_word_vectors.p", "rb"))
    id2token = list(word_vectors.keys())
    token2id = dict(zip(word_vectors, range(2,2+len(word_vectors)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return word_vectors, token2id, id2token

In [6]:
def convert_labels_to_integers(data_label):
    for i in range(len(data_label)):
        if data_label[i] == "contradiction":
            data_label[i] = 0
        elif data_label[i] == "entailment":
            data_label[i] = 1
        elif data_label[i] == "neutral":
            data_label[i] = 2
    return data_label

In [7]:
def verify_order(sent1_data, sent2_data, data_label):
    i = random.randint(1, len(sent1_data))
    print(sent1_data[i])
    print(sent2_data[i])
    print(data_label[i])

In [8]:
# Word tokenize each entry in a list of sentences
def tokenize(sentence_list):
    return [word_tokenize(sentence_list[i]) for i in range(len(sentence_list))]

In [9]:
# "one-hot encode": convert each token to id in vocabulary vector (token2id)
def token2index_dataset(tokens_data):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data 

### Creating vocabulary & embedding matrix from FastText

In [10]:
word_vectors, token2id, id2token = build_vocab()

In [11]:
_weights = np.array(list(word_vectors.values()))
pad_vec = np.zeros((1, 300))
unk_vec = np.random.randn(1, 300) * 0.01
pad_unk_vecs = np.vstack((pad_vec, unk_vec))
_WEIGHTS = np.vstack((pad_unk_vecs, _weights))
_WEIGHTS.shape

(50002, 300)

### Function to pre-process data for TwoSentenceModel
#### Shuffle, word tokenize, one-hot index into vocabulary

In [12]:
def data_pipeline(sent1s, sent2s, labels, verify=True):
    labels = convert_labels_to_integers(labels)
    seed = random.randint(1, 100)
    print("Random seed for shuffling: {}".format(seed))
    random.Random(seed).shuffle(sent1s)
    random.Random(seed).shuffle(sent2s)
    random.Random(seed).shuffle(labels)
    
    print("\nVerifying that the data and label match after shuffling")
    if verify:
        verify_order(sent1s, sent2s, labels)
        verify_order(sent1s, sent2s, labels)
          
    print("\nTokenizing sentence 1 list...")    
    sent1s_tokenized = tokenize(sent1s)
    print("done!")
    print("\nTokenizing sentence 2 list... ")  
    sent2s_tokenized = tokenize(sent2s)
    print("done!")
    
    print("\nOne-hot encoding words for sentence 1 list...")  
    sent1s_indices = token2index_dataset(sent1s_tokenized)
    print("done!")
    print("\nOne-hot encoding words for sentence 2 list...")  
    sent2s_indices = token2index_dataset(sent2s_tokenized)
    print("done!")
    
    return (sent1s_indices, sent2s_indices, labels)

### DataLoader

In [13]:
# MAX_SENTENCE_LENGTH = 30

import numpy as np
import torch
from torch.utils.data import Dataset

class TwoSentencesDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    """
    
    def __init__(self, sent1_data_list, sent2_data_list, target_list):
        """
        @param sent1_data_list: list of sentence1's (index matches sentence2's and target_list below)
        @param sent2_data_list: list of sentence2's
        @param target_list: list of correct labels

        """
        self.sent1_data_list = sent1_data_list
        self.sent2_data_list = sent2_data_list
        self.target_list = target_list
        assert (len(self.sent1_data_list) == len(self.target_list) and len(self.sent2_data_list) == len(self.target_list))

    def __len__(self):
        return len(self.sent1_data_list)
        
    def __getitem__(self, key):
        ###
        ### Returns [[sentence, 1, tokens], [sentence, 2, tokens]]
        ###
        """
        Triggered when you call dataset[i]
        """
        sent1_tokens_idx = self.sent1_data_list[key][:MAX_SENTENCE_LENGTH]
        sent2_tokens_idx = self.sent2_data_list[key][:MAX_SENTENCE_LENGTH]
        combined_tokens_idx = [sent1_tokens_idx, sent2_tokens_idx]
        label = self.target_list[key]
        return [combined_tokens_idx, len(sent1_tokens_idx), len(sent2_tokens_idx), label]

def twosentences_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    sent1_data_list = []
    sent2_data_list = []
    sent1_length_list = []
    sent2_length_list = []
    label_list = []
    combined_data_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[3])
        sent1_length_list.append(datum[1])
        sent2_length_list.append(datum[2])
    # padding
    for datum in batch:
        padded_vec_1 = np.pad(np.array(datum[0][0]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        padded_vec_2 = np.pad(np.array(datum[0][1]), pad_width=((0,MAX_SENTENCE_LENGTH-datum[2])), 
                                mode="constant", constant_values=0)
        combined_data_list.append([padded_vec_1, padded_vec_2])
    return [torch.from_numpy(np.array(combined_data_list)), 
            torch.LongTensor(sent1_length_list), torch.LongTensor(sent2_length_list), torch.LongTensor(label_list)]

### Train dataset creation

In [14]:
snli_train = pd.read_csv('snli_train.tsv', sep='\t')
TRAIN_SIZE = 100000

sent1_data = list(snli_train["sentence1"])[:TRAIN_SIZE]
sent2_data = list(snli_train["sentence2"])[:TRAIN_SIZE]
data_label = list(snli_train["label"])[:TRAIN_SIZE]
print("Size of training data: {}".format(len(sent1_data)))

Size of training data: 100000


In [15]:
sent1_train_indices, sent2_train_indices, train_label = data_pipeline(sent1_data, sent2_data, data_label)
train_dataset = TwoSentencesDataset(sent1_train_indices, sent2_train_indices, train_label)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )
print("Finished creating train_loader.")

Random seed for shuffling: 91

Verifying that the data and label match after shuffling
An old man looks at people shopping .
An old man is watching people shop .
1
A brown dog is running along a beach .
He is wagging his tail .
2

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!
Finished creating train_loader.


### Val dataset creation

In [16]:
snli_val = pd.read_csv('snli_val.tsv', sep='\t')
sent1_val = list(snli_val["sentence1"])
sent2_val = list(snli_val["sentence2"])
val_label = list(snli_val["label"])
print("Size of val data: {}".format(len(sent1_val)))

Size of val data: 1000


In [17]:
sent1_val_indices, sent2_val_indices, val_label = data_pipeline(sent1_val, sent2_val, val_label)
val_dataset = TwoSentencesDataset(sent1_val_indices, sent2_val_indices, val_label)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE, 
                                           collate_fn=twosentences_collate_func,
                                           #shuffle=True
                                          )

Random seed for shuffling: 1

Verifying that the data and label match after shuffling
Two men , one with a black shirt and the other with a white shirt , are kicking each other without making contact .
Two men are fighting over a girl
2
Three children run on a sidewalk with a river on one side and a wall on the other .
Three children are swimming next to the sidewalk .
0

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!


In [18]:
pd.Series([len(x) for x in snli_train['sentence1']]).describe()['75%']
MAX_SENTENCE_LENGTH = 40

### The Model

In [22]:
class TwoSentenceModel(nn.Module):
    def __init__(self, hidden_size, num_layers, num_classes, emb_size = 300):
        # RNN Accepts the following hyperparams:
        # emb_size: Embedding Size
        # hidden_size: Hidden Size of layer in RNN
        # num_layers: number of layers in RNN
        # num_classes: number of output classes
        # vocab_size: vocabulary size
        super(TwoSentenceModel, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        weight = torch.FloatTensor(_WEIGHTS)
        self.embedding = nn.Embedding.from_pretrained(weight)
#         self.rnn = nn.RNN(emb_size, hidden_size, num_layers, batch_first=True)
#         self.linear = nn.Linear(2*hidden_size, num_classes)
        # TRYING GRU, UNCOMMENT Below if doing GRU
        self.rnn = nn.GRU(emb_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.linear1 = nn.Linear(2*hidden_size, 100)
        self.linear2 = nn.Linear(100, num_classes)
        self.dropout = nn.Dropout(0.2)

    def init_hidden(self, batch_size):
        # Function initializes the activation of recurrent neural net at timestep 0
        # Needs to be in format (num_layers, batch_size, hidden_size)
        return torch.randn(2, batch_size*2, self.hidden_size).to(device)

    def forward(self, x, sent1_lengths, sent2_lengths):
        # reset hidden state
        batch_size = x.size()[0]
                
        s1lengths = list(sent1_lengths)
        s2lengths = list(sent2_lengths)
        ordered_slengths = s1lengths + s2lengths

        reverse_sorted_indices = [x for _, x in sorted(zip(ordered_slengths, range(len(ordered_slengths))), reverse=True)]
        reverse_sorted_lengths = [x for x, _ in sorted(zip(ordered_slengths, range(len(ordered_slengths))), reverse=True)]
        reverse_sorted_lengths = np.array(reverse_sorted_lengths)
        
        sent1s = x[:, 0, :]
        sent2s = x[:, 1, :]
        ordered_sents = torch.cat([sent1s, sent2s], dim=0).to(device)
        reverse_sorted_data = torch.index_select(ordered_sents, 0, torch.tensor(reverse_sorted_indices).to(device))
        
        # get embedding
        embed = self.embedding(reverse_sorted_data)
        
        self.hidden = self.init_hidden(batch_size)

        # pack padded sequence
        embed = torch.nn.utils.rnn.pack_padded_sequence(embed, reverse_sorted_lengths, batch_first=True)
              
        # fprop though RNN
        rnn_out, self.hidden = self.rnn(embed, self.hidden)
                
        ### MATCHING BACK
        change_it_back = [x for _, x in sorted(zip(reverse_sorted_indices, range(len(reverse_sorted_indices))))]
        self.hidden = torch.index_select(self.hidden, 1, torch.LongTensor(change_it_back).to(device)) 
        
        # 2 by 64 by 250. back in the right order that it came in.        
#         hidden_sent1s = self.hidden[0, 0:batch_size, :]
#         hidden_sent2s = self.hidden[0, batch_size:, :]
              
        ### GRU stuff
        hidden_sent1s = torch.cat([self.hidden[0, 0:batch_size, :], self.hidden[1, 0:batch_size, :]], dim=1)
        hidden_sent2s = torch.cat([self.hidden[0, batch_size:, :], self.hidden[1, batch_size:, :]], dim=1)
        
#         concatenation of encoded sentences
#         linear1 = self.linear1(torch.cat([hidden_sent1s, hidden_sent2s], dim=1))
#         addition of encoded sentences
#         linear1 = self.linear1(torch.tensor(hidden_sent1s) + torch.tensor(hidden_sent2s))
#         element-wise multiplication of encoded sentences
        linear1 = self.linear1(torch.tensor(hidden_sent1s)*torch.tensor(hidden_sent2s))
        linear1 = F.relu(linear1.contiguous().view(-1, linear1.size(-1))).view(linear1.shape)   
#         linear1 = self.dropout(linear1)
        logits = self.linear2(linear1)
        return logits

In [23]:
# Function for testing the model
def test_model(loader, model):
    """
    Helper function that tests the model's performance on a dataset
    """
    correct = 0
    total = 0
    model.eval()
    for (data, sent1_lengths, sent2_lengths, labels) in loader:
        data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.to(device), sent1_lengths.to(device), sent2_lengths.to(device), labels.to(device)
        outputs = F.softmax(model(data_batch, sent1_length_batch, sent2_length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        labels = labels.to(device)
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

def train_model(model, lr = 0.001, num_epochs = 7, criterion = nn.CrossEntropyLoss()):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
    max_val_acc = 0
    for epoch in range(num_epochs):
        for i, (data, sent1_lengths, sent2_lengths, labels) in enumerate(train_loader):
            model.train()
            data_batch, sent1_length_batch, sent2_length_batch, label_batch = data.to(device), sent1_lengths.to(device), sent2_lengths.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data_batch, sent1_length_batch, sent2_length_batch)
            loss = criterion(outputs, label_batch)
            
            loss.backward()
            optimizer.step()
            # validate every 100 iterations
            if i > 0 and i % 100 == 0:
                # validate
                val_acc = test_model(val_loader, model)
                if val_acc > max_val_acc:
                    max_val_acc = val_acc
                print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), val_acc))
                print('Epoch: [{}/{}], Step: [{}/{}], Training Loss: {}'.format( 
                           epoch+1, num_epochs, i+1, len(train_loader), loss))
                
    print("Max Validation Accuracy: {}".format(max_val_acc))
    return max_val_acc

In [24]:
model = TwoSentenceModel(emb_size = 300, hidden_size=300, num_layers=1, num_classes=3).to(device)
train_model(model)

Epoch: [1/7], Step: [101/3125], Validation Acc: 33.9
Epoch: [1/7], Step: [101/3125], Training Loss: 1.107649326324463
Epoch: [1/7], Step: [201/3125], Validation Acc: 37.2
Epoch: [1/7], Step: [201/3125], Training Loss: 1.080571174621582
Epoch: [1/7], Step: [301/3125], Validation Acc: 37.0
Epoch: [1/7], Step: [301/3125], Training Loss: 1.116148591041565
Epoch: [1/7], Step: [401/3125], Validation Acc: 38.7
Epoch: [1/7], Step: [401/3125], Training Loss: 1.0383262634277344
Epoch: [1/7], Step: [501/3125], Validation Acc: 37.5
Epoch: [1/7], Step: [501/3125], Training Loss: 1.0529433488845825
Epoch: [1/7], Step: [601/3125], Validation Acc: 40.6
Epoch: [1/7], Step: [601/3125], Training Loss: 1.0518783330917358
Epoch: [1/7], Step: [701/3125], Validation Acc: 42.8
Epoch: [1/7], Step: [701/3125], Training Loss: 1.1364549398422241
Epoch: [1/7], Step: [801/3125], Validation Acc: 42.5
Epoch: [1/7], Step: [801/3125], Training Loss: 1.0270627737045288
Epoch: [1/7], Step: [901/3125], Validation Acc: 42.

Epoch: [3/7], Step: [801/3125], Validation Acc: 62.9
Epoch: [3/7], Step: [801/3125], Training Loss: 0.7256885766983032
Epoch: [3/7], Step: [901/3125], Validation Acc: 64.0
Epoch: [3/7], Step: [901/3125], Training Loss: 0.7802861332893372
Epoch: [3/7], Step: [1001/3125], Validation Acc: 64.2
Epoch: [3/7], Step: [1001/3125], Training Loss: 0.7330567240715027
Epoch: [3/7], Step: [1101/3125], Validation Acc: 62.7
Epoch: [3/7], Step: [1101/3125], Training Loss: 0.9180753231048584
Epoch: [3/7], Step: [1201/3125], Validation Acc: 63.1
Epoch: [3/7], Step: [1201/3125], Training Loss: 0.7830043435096741
Epoch: [3/7], Step: [1301/3125], Validation Acc: 64.2
Epoch: [3/7], Step: [1301/3125], Training Loss: 0.7513207197189331
Epoch: [3/7], Step: [1401/3125], Validation Acc: 64.6
Epoch: [3/7], Step: [1401/3125], Training Loss: 0.7049599885940552
Epoch: [3/7], Step: [1501/3125], Validation Acc: 64.7
Epoch: [3/7], Step: [1501/3125], Training Loss: 0.7262147068977356
Epoch: [3/7], Step: [1601/3125], Val

Epoch: [5/7], Step: [1501/3125], Validation Acc: 68.7
Epoch: [5/7], Step: [1501/3125], Training Loss: 0.6363848447799683
Epoch: [5/7], Step: [1601/3125], Validation Acc: 65.9
Epoch: [5/7], Step: [1601/3125], Training Loss: 0.5829861760139465
Epoch: [5/7], Step: [1701/3125], Validation Acc: 67.7
Epoch: [5/7], Step: [1701/3125], Training Loss: 0.5647152662277222
Epoch: [5/7], Step: [1801/3125], Validation Acc: 66.9
Epoch: [5/7], Step: [1801/3125], Training Loss: 0.6764464974403381
Epoch: [5/7], Step: [1901/3125], Validation Acc: 65.8
Epoch: [5/7], Step: [1901/3125], Training Loss: 0.6128199100494385
Epoch: [5/7], Step: [2001/3125], Validation Acc: 66.9
Epoch: [5/7], Step: [2001/3125], Training Loss: 0.6860468983650208
Epoch: [5/7], Step: [2101/3125], Validation Acc: 68.2
Epoch: [5/7], Step: [2101/3125], Training Loss: 0.6453467607498169
Epoch: [5/7], Step: [2201/3125], Validation Acc: 68.4
Epoch: [5/7], Step: [2201/3125], Training Loss: 0.5940616130828857
Epoch: [5/7], Step: [2301/3125],

Epoch: [7/7], Step: [2201/3125], Validation Acc: 68.8
Epoch: [7/7], Step: [2201/3125], Training Loss: 0.5369063019752502
Epoch: [7/7], Step: [2301/3125], Validation Acc: 70.0
Epoch: [7/7], Step: [2301/3125], Training Loss: 0.5517008304595947
Epoch: [7/7], Step: [2401/3125], Validation Acc: 68.0
Epoch: [7/7], Step: [2401/3125], Training Loss: 0.6681950092315674
Epoch: [7/7], Step: [2501/3125], Validation Acc: 68.5
Epoch: [7/7], Step: [2501/3125], Training Loss: 0.5436170697212219
Epoch: [7/7], Step: [2601/3125], Validation Acc: 68.2
Epoch: [7/7], Step: [2601/3125], Training Loss: 0.5800732970237732
Epoch: [7/7], Step: [2701/3125], Validation Acc: 68.1
Epoch: [7/7], Step: [2701/3125], Training Loss: 0.5750572085380554
Epoch: [7/7], Step: [2801/3125], Validation Acc: 68.4
Epoch: [7/7], Step: [2801/3125], Training Loss: 0.7890423536300659
Epoch: [7/7], Step: [2901/3125], Validation Acc: 67.0
Epoch: [7/7], Step: [2901/3125], Training Loss: 0.38154488801956177
Epoch: [7/7], Step: [3001/3125]

70.2

In [25]:
# torch.save(model.state_dict(), "/home/vrajiv/rnn-cnn-natural-language-inference/best_rnn")

### Evaluating on MNLI

In [26]:
mnli_val = pd.read_csv('mnli_val.tsv', sep='\t')

In [27]:
sentence_length_75 = pd.Series([len(x) for x in mnli_val['sentence1']]).describe()['75%']
print("75th percentile for sentence length (in characters): {}".format(sentence_length_75))
MAX_SENTENCE_LENGTH = 40

75th percentile for sentence length (in characters): 151.0


#### Build dictionary of (sent1, sent2, label) data, by genre

In [28]:
mnli_val_dict = {}
for x in mnli_val['genre'].unique():
    filtered = mnli_val[mnli_val['genre'] == x]
    mnli_val_dict[x] = {}
    mnli_val_dict[x]["sent1s"] = list(filtered["sentence1"])
    mnli_val_dict[x]["sent2s"] = list(filtered["sentence2"])
    mnli_val_dict[x]["label"] = convert_labels_to_integers(list(filtered["label"]))

In [29]:
for x in mnli_val_dict.keys():
    print(len(mnli_val_dict[x]["sent1s"]))
    print(np.unique(mnli_val_dict[x]["label"]))
    
# quick verify
verify_order(mnli_val_dict['travel']["sent1s"], mnli_val_dict['travel']["sent2s"], mnli_val_dict['travel']["label"])

995
[0 1 2]
1005
[0 1 2]
1002
[0 1 2]
1016
[0 1 2]
982
[0 1 2]
Now open political debate flourished , especially in Calcutta where Karl Marx was much appreciated .
Now political debate died down in Calcutta especially , where Karl Marx was hated .
0


In [30]:
# for each genre, build validation set and evaluate on it. 
cnn_results = {}
model = TwoSentenceModel(emb_size = 300, hidden_size=300, num_layers=1, num_classes=3).to(device)
model.load_state_dict(torch.load("/home/vrajiv/rnn-cnn-natural-language-inference/best_rnn"))
model.eval()
for genre in mnli_val_dict.keys():
    sent1_val_indices, sent2_val_indices, val_label = data_pipeline(mnli_val_dict[genre]["sent1s"], 
                                                                    mnli_val_dict[genre]["sent2s"], 
                                                                    mnli_val_dict[genre]["label"])
    val_dataset = TwoSentencesDataset(sent1_val_indices, sent2_val_indices, val_label)
    val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                             batch_size=BATCH_SIZE, 
                                             collate_fn=twosentences_collate_func,
                                             #shuffle=True
                                             )
    cnn_results[genre] = test_model(val_loader, model)

for genre in mnli_val_dict.keys():
    print("Genre {} has validation accuracy: {}".format(genre, cnn_results[genre]))
    

Random seed for shuffling: 95

Verifying that the data and label match after shuffling
'You burned down my house . '
'You used matches and gasoline to commit arson . '
2
I sha n't stop you . ''
You can stop .
0

Tokenizing sentence 1 list...
done!

Tokenizing sentence 2 list... 
done!

One-hot encoding words for sentence 1 list...
done!

One-hot encoding words for sentence 2 list...
done!
Random seed for shuffling: 79

Verifying that the data and label match after shuffling
maybe adult literacy maybe you know composition writing maybe you know uh volunteering you know on a tutor line or though the even through the elementary schools for help with homework or the other part of me says is God i 've had enough kids do i really
maybe I could volunteer to help coach sports since I 've helped all my children be successful in sports
0
that 's right you can work yourself to death well i 'm sorry to hear your color did n't come out so good over the weekend
I 'm sorry it did n't turn out as plan

In [120]:
sample_sent1 = ['How', "are", "you", "doing", "my", "main", "man"]
sample_sent2 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]
sample_token_data = [sample_sent1, sample_sent2]

indices_data = torch.from_numpy(np.array(token2index_dataset(sample_token_data)))
model.forward(indices_data, lengths = torch.from_numpy(np.array([7, 7])))

TypeError: forward() got an unexpected keyword argument 'lengths'

In [ ]:
sample_sent1 = ['How', "are", "you", "doing", "my", "main", "man", "pad"]
sample_sent2 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]

sample_sent3 = ['How', "are", "you", "doing", "my", "main", "man", "pad"]
sample_sent4 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]

sample_sent5 = ['How', "are", "you", "doing", "my", "main", "man", "pad"]
sample_sent6 = ['I', "am", "building", "a", "bruh", "hehe", "classifier"]

sample_combined_list = []
sample_combined_list.append([sample_sent1, sample_sent2])
sample_combined_list.append([sample_sent3, sample_sent4])
sample_combined_list.append([sample_sent5, sample_sent6])


np.array(sample_combined_list).shape

In [ ]:
for i, (data, sent1_lengths, sent2_lengths, labels) in enumerate(train_loader):
    print("DATA\n")
    print(data.size())
    print (data)
    print(sent1_lengths)
    print(sent2_lengths)
    print (labels)
    break

In [ ]:
###
### Initialize unk vector to * 0.01 and 0.001.  
###

###
### Sort descending before doing pack_padded_sequence
###

###
### Sort all training sentences by descending order. 
### Feed it into the batch. 
### Run your model through each of them to get hidden outputs
### Match them back again to right order to calculate loss. 
###

###
### Freeze the embedding matrix weights so they don't train. 
###